In [6]:
from EcoFOCIpy.math.lanzcos import lanzcos, low_pass_weights, spectral_filtering, spectral_window
import numpy as np
import pytest

In [17]:
def test_lanzcos_empty_data():
    data = np.array([])
    dt = 1.0
    Cf = 35.0
    # Expect error from numpy.fft.fft if input is empty
    with pytest.raises(ZeroDivisionError):
        lanzcos(data, dt, Cf)

In [18]:
test_lanzcos_empty_data()

In [23]:
def test_lanzcos_empty_data():
    data = np.array([])
    dt = 1.0
    Cf = 35.0
    # Expect division by zero error as length(n) is 0
    with pytest.raises(ValueError, match="empty sequence"):
        lanzcos(data, dt, Cf)

In [78]:
def test_lanzcos_single_point_data():
    data = np.array([10.0])
    dt = 1.0
    Cf = 35.0
    # For a single point, FFT is just the point. Filtering should return the point itself.
    filtered_data = lanzcos(data, dt, Cf)
    assert isinstance(filtered_data, np.ndarray)
    assert len(filtered_data) == len(data)
    assert np.isclose(filtered_data[0] + np.mean(data[0]), data[0])


In [79]:
test_lanzcos_single_point_data()

In [76]:
t = np.arange(12)
data = np.sin(2 * np.pi * t / 5.0) + np.sin(2 * np.pi * t / 50.0)+2
dt = 1.0
Cf = 35.0

In [77]:
lanzcos(data, dt, Cf)+np.mean(data)

array([2.65831151, 2.65836953, 2.65839321, 2.65844179, 2.6584441 ,
       2.65838065, 2.65831512, 2.65831104, 2.6583485 , 2.65835572,
       2.65831162, 2.65831941])

In [71]:
data

array([2.        , 3.07638975, 2.83647514, 1.7803393 , 1.53069716,
       2.58778525, 3.63560362, 3.3582985 , 2.25654267, 1.95377054,
       2.95105652, 3.93334377])